<a href="https://colab.research.google.com/github/theanhvu1998/VuTheAnh_Assignment/blob/master/Copy_of_Week5_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Bag of Words Meets Bags of Popcorn](https://www.kaggle.com/c/word2vec-nlp-tutorial/data)
======

## Data Set

The labeled data set consists of 50,000 IMDB movie reviews, specially selected for sentiment analysis. The sentiment of reviews is binary, meaning the IMDB rating < 5 results in a sentiment score of 0, and rating >=7 have a sentiment score of 1. No individual movie has more than 30 reviews. The 25,000 review labeled training set does not include any of the same movies as the 25,000 review test set. In addition, there are another 50,000 IMDB reviews provided without any rating labels.

## File descriptions

labeledTrainData - The labeled training set. The file is tab-delimited and has a header row followed by 25,000 rows containing an id, sentiment, and text for each review.
## Data fields

* id - Unique ID of each review
* sentiment - Sentiment of the review; 1 for positive reviews and 0 for negative reviews
* review - Text of the review

## Objective
Objective of this dataset is base on **review** we predict **sentiment** (positive or negative) so X is **review** column and y is **sentiment** column

## 1. Load Dataset

Let's first of all have a look at the data. You can download the file `labeledTrainData.tsv` on the [Kaggle website of the competition](https://www.kaggle.com/c/word2vec-nlp-tutorial/data), or on our [Google Drive](https://drive.google.com/file/d/1a1Lyn7ihikk3klAX26fgO3YsGdWHWoK5/view?usp=sharing)


In [11]:
# Import pandas, numpy
import pandas as pd
import numpy as np

In [12]:
# Read dataset with extra params sep='\t', encoding="latin-1"
sentiment = pd.read_csv('https://raw.githubusercontent.com/dhminh1024/practice_datasets/master/twitter_sentiment_analysis.csv', encoding='latin-1')
sentiment.sample(5)

,ItemID,Sentiment,SentimentText
59461,59473,0,"@bekasaur i've told her too, i want my kitty ..."
45026,45038,0,@annietrevino i miss you sooo much come ba...
52337,52349,1,@Asian_Ladies hello there how's Vegas?
52435,52447,1,@askdebra Thanks Glad you liked the pics.
38033,38045,1,@AmberCharlotte Nice background lovely


## 2. Preprocessing

In [13]:
# stop words
import nltk
nltk.download('stopwords')
# Removing stopwords
def remove_stop_words(text):
  return [item for item in text if item not in stop_words]
   # Your code here

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
# Removing special characters and "trash"
import re
def preprocessor(text):
    # Remove HTML markup
    # text = None # Your code here
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Your code here
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    # Your code here
    
    return text

In [22]:
# tokenizer and stemming
# tokenizer: to break down our twits in individual words
# stemming: reducing a word to its root
from nltk.stem import PorterStemmer
# Your code here

porter = PorterStemmer()

def tokenizer_porter(text):
  return [porter.stem(word) for word in text.split()]
   # Your code here

In [23]:
# split the dataset in train and test
# Your code here
sentiment["CleanSentimentText"] = sentiment["SentimentText"].apply(preprocessor)
sentiment
from nltk.corpus import stopwords
stop_words = stopwords.words('english')


## 3. Create Model and Train 

Using **Pipeline** to concat **tfidf** step and **LogisticRegression** step

In [48]:
# Import Pipeline, LogisticRegression, TfidfVectorizer
from sklearn.model_selection import train_test_split

X = sentiment['SentimentText']
y = sentiment['Sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=102)

In [49]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop_words,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)

In [50]:
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])
clf.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x7f9bf55372f0>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', '...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x7f9c072c9488>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
         

In [92]:
p_predict = clf.predict_proba(X_test)
f_predict = clf.predict(X_test)
p_predict = pd.DataFrame(p_predict).reindex().rename(columns={0: "Negative",1: "Positive"}, errors="raise")
f_predict = pd.DataFrame(f_predict).rename(columns={0: "Predict"}, errors="raise")
predict = pd.concat([X_test.reset_index(), y_test.reset_index(),f_predict.reindex(),p_predict], axis=1)
predict.drop("index", axis=1)

,SentimentText,Sentiment,Predict,Negative,Positive
0,@bambamboogie hate to disappoint but HC wasnt ...,0,0,0.958294,0.041706
1,@cianaftw shuddup,0,1,0.305323,0.694677
2,@ABenton still waiting for the BW20 AD Networ...,1,1,0.268569,0.731431
3,@CassieEsbaugh Why good morning Rye Bread Wha...,1,1,0.261089,0.738911
4,Why he gotta look like that?,0,0,0.783496,0.216504
...,...,...,...,...,...
19993,@butterflie so glad I got to see you! &amp; EX...,1,1,0.141213,0.858787
19994,"@AuntSay I hope so, the thought of never seein...",1,0,0.968300,0.031700
19995,@AlanCarr when's your chat show starting? And ...,1,1,0.403227,0.596773
19996,@Arie_R_J where yu att??,0,1,0.435781,0.564219


## 4. Evaluate Model

In [93]:
# Using Test dataset to evaluate model
# classification_report
# confusion matrix
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

# Now apply those above metrics to evaluate your model
# Your code here
predictions = clf.predict(X_test)
print('accuracy:',accuracy_score(y_test,predictions))
print('confusion matrix:\n',confusion_matrix(y_test,predictions))
print('classification report:\n',classification_report(y_test,predictions))

accuracy: 0.7525752575257526
confusion matrix:
 [[5828 2877]
 [2071 9222]]
classification report:
               precision    recall  f1-score   support

           0       0.74      0.67      0.70      8705
           1       0.76      0.82      0.79     11293

    accuracy                           0.75     19998
   macro avg       0.75      0.74      0.75     19998
weighted avg       0.75      0.75      0.75     19998



## 5. Export Model 

In [94]:
# Using pickle to export our trained model
import pickle
import os

pickle.dump(clf, open('logisticRegression.pkl', 'wb'))

In [96]:
with open('logisticRegression.pkl', 'rb') as model:
    reload_model = pickle.load(model)
preds = reload_model.predict_proba(X_test)

p_predict = clf.predict_proba(X_test)
f_predict = clf.predict(X_test)
p_predict = pd.DataFrame(p_predict).reindex().rename(columns={0: "Negative",1: "Positive"}, errors="raise")
f_predict = pd.DataFrame(f_predict).rename(columns={0: "Predict"}, errors="raise")
predict = pd.concat([X_test.reset_index(), y_test.reset_index(),f_predict.reindex(),p_predict], axis=1)
predict.drop("index", axis=1)

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


,SentimentText,Sentiment,Predict,Negative,Positive
0,@bambamboogie hate to disappoint but HC wasnt ...,0,0,0.958294,0.041706
1,@cianaftw shuddup,0,1,0.305323,0.694677
2,@ABenton still waiting for the BW20 AD Networ...,1,1,0.268569,0.731431
3,@CassieEsbaugh Why good morning Rye Bread Wha...,1,1,0.261089,0.738911
4,Why he gotta look like that?,0,0,0.783496,0.216504
...,...,...,...,...,...
19993,@butterflie so glad I got to see you! &amp; EX...,1,1,0.141213,0.858787
19994,"@AuntSay I hope so, the thought of never seein...",1,0,0.968300,0.031700
19995,@AlanCarr when's your chat show starting? And ...,1,1,0.403227,0.596773
19996,@Arie_R_J where yu att??,0,1,0.435781,0.564219
